In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import os

# Initialize a list to store the accuracies for each group
accuracies = []

# Loop through group numbers from 1 to 5
for group_number in range(1, 6):
    
    # Load the CSV files from the specified group
    sdl_class_rdoc_path = f'0.class_document/{group_number}/sdl_class_rdoc.csv'
    test_path = f'../../data_preprocess/dataset/{group_number}/test.csv'
    
    # Check if test file exists, if not, skip this iteration
    if not os.path.exists(test_path):
        print(f"test file for Group {group_number} does not exist. Skipping...")
        continue
    
    sdl_class_rdoc_csv = pd.read_csv(sdl_class_rdoc_path, low_memory=False)
    test_csv = pd.read_csv(test_path, low_memory=False)
    
    # Replace NaN values with empty strings in relevant columns
    sdl_class_rdoc_csv['tag_description'] = sdl_class_rdoc_csv['tag_description'].fillna('')
    test_csv['tag_description'] = test_csv['tag_description'].fillna('')
    
    # Initialize new columns in test_csv
    test_csv['c_thing'] = ''
    test_csv['c_property'] = ''
    test_csv['c_score'] = ''
    test_csv['c_duplicate'] = 0  # Initialize c_duplicate to store duplicate counts
    
    # Combine both sdl_class_rdoc and test CSVs tag_descriptions for TF-IDF Vectorizer training
    combined_tag_descriptions = sdl_class_rdoc_csv['tag_description'].tolist() + test_csv['tag_description'].tolist()
    
    # Create a TF-IDF Vectorizer
    vectorizer = TfidfVectorizer(
        token_pattern=r'\S+',
        ngram_range=(1, 6),  # Use ngrams from 1 to 6
    )
    
    # Fit the TF-IDF vectorizer on the combined tag_descriptions
    vectorizer.fit(combined_tag_descriptions)
    
    # Transform both sdl_class_rdoc and test CSVs into TF-IDF matrices
    sdl_class_rdoc_tfidf_matrix = vectorizer.transform(sdl_class_rdoc_csv['tag_description'])
    test_tfidf_matrix = vectorizer.transform(test_csv['tag_description'])
    
    # Calculate cosine similarity between test and class-level sdl_class_rdoc vectors
    similarity_matrix = cosine_similarity(test_tfidf_matrix, sdl_class_rdoc_tfidf_matrix)
    
    # Find the most similar class-level tag_description for each test description
    most_similar_indices = similarity_matrix.argmax(axis=1)
    most_similar_scores = similarity_matrix.max(axis=1)
    
    # Assign the corresponding thing, property, and similarity score to the test CSV
    test_csv['c_thing'] = sdl_class_rdoc_csv.iloc[most_similar_indices]['thing'].values
    test_csv['c_property'] = sdl_class_rdoc_csv.iloc[most_similar_indices]['property'].values
    test_csv['c_score'] = most_similar_scores
    
    # Check if the predicted 'c_thing' and 'c_property' match the actual 'thing' and 'property'
    test_csv['cthing_correct'] = test_csv['thing'] == test_csv['c_thing']
    test_csv['cproperty_correct'] = test_csv['property'] == test_csv['c_property']
    test_csv['ctp_correct'] = test_csv['cthing_correct'] & test_csv['cproperty_correct']
    
    # Calculate accuracy based only on MDM = True
    mdm_true_count = len(test_csv[test_csv['MDM'] == True])
    accuracy = (test_csv['ctp_correct'].sum() / mdm_true_count) * 100
    accuracies.append(accuracy)
    
    print(f"Accuracy (MDM=True) for Group {group_number}: {accuracy:.2f}%")
    
    # Specify output file paths
    output_path = f'0.class_document/{group_number}/test_p_c.csv'
    test_csv.to_csv(output_path, index=False, encoding='utf-8-sig')
    
    # Filter for rows where MDM is True and ctp_correct is False
    false_positive_rows = test_csv[(test_csv['MDM'] == True) & (test_csv['ctp_correct'] == False)]
    
    # Save false positives to a separate file
    fp_output_path = f'0.class_document/{group_number}/fp_class.csv'
    false_positive_rows.to_csv(fp_output_path, index=False, encoding='utf-8-sig')

# Calculate and print the average accuracy across all groups
average_accuracy = sum(accuracies) / len(accuracies)
print(f"Average Accuracy (MDM=True) across all groups: {average_accuracy:.2f}%")


Accuracy (MDM=True) for Group 1: 79.41%
Accuracy (MDM=True) for Group 2: 79.32%
Accuracy (MDM=True) for Group 3: 82.49%
Accuracy (MDM=True) for Group 4: 85.61%
Accuracy (MDM=True) for Group 5: 79.72%
Average Accuracy (MDM=True) across all groups: 81.31%
